# Fusion baseline Model (GOES + Ground)

## Libraries

In [ ]:
from __future__ import annotations
import os, re, json, math, warnings
from pathlib import Path
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional


import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

import rioxarray # noqa: F401

from satpy import Scene
from satpy.writers import get_enhanced_image

import tensorflow as tf
from tensorflow.keras import layers, models

2025-09-11 07:38:43.120238: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-11 07:38:43.125691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757594323.131939  336635 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757594323.134004  336635 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-11 07:38:43.141095: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Config

In [ ]:
@dataclass
class Config:
    product: str # 'DSRF' or 'MCMIPF'
    dsrf_dir: Path # directory with DSRF .nc (ignored if product='MCMIPF')
    mcmipf_dir: Path # directory with MCMIPF .nc (ignored if product='DSRF')
    parquet_train: Path # ground/tabular train parquet
    parquet_val: Path # ground/tabular val parquet
    parquet_test: Path # ground/tabular test parquet
    target_col: str # e.g., 'y_ghi_h6'
    feature_cols: Optional[List[str]] = None # if None → numeric columns minus target


    aoi_bbox: Optional[Tuple[float,float,float,float]] = None # (min_lon, min_lat, max_lon, max_lat)
    res_km: float = 2.0 # output grid resolution (approx degrees will be inferred)


    seq_len: int = 12 # timesteps for image & tabular sequences
    lead_steps: int = 0 # if your target is offset vs last image; keep 0 if target aligned to last step


    batch_size: int = 16
    epochs: int = 50
    lr: float = 1e-3


    out_dir: Path = Path("../models") # usa OUT_DIR conocido

## File indexing

In [ ]:
_TS_RGX = re.compile(r"(20\d{6})[_T]?(\d{4})") # matches YYYYMMDD[_T]HHMM




def ts_from_name(p: Path) -> Optional[pd.Timestamp]:
    m = _TS_RGX.search(p.name)
    if not m:
        return None
    ymd, hm = m.groups()
    try:
        return pd.to_datetime(ymd + hm, format="%Y%m%d%H%M", utc=True)
    except Exception:
        return None




def list_nc_by_ts(root: Path) -> pd.Series:
    files = sorted([p for p in Path(root).glob("**/*.nc")])
    pairs = [(ts_from_name(p), p) for p in files]
    pairs = [(t, p) for (t, p) in pairs if t is not None]
    if not pairs:
        raise FileNotFoundError(f"No .nc files with timestamp pattern under {root}")
    s = pd.Series({t: p for t, p in pairs}).sort_index()
    s.index.name = "ts"
    return s

## Helpers

### Reader

In [ ]:
def _crop_latlon_da(da: xr.DataArray, bbox: Tuple[float,float,float,float]) -> xr.DataArray:
    min_lon, min_lat, max_lon, max_lat = bbox
    if 'lon' in da.coords and 'lat' in da.coords:
        da = da.sortby('lon').sortby('lat')
        da = da.sel(lon=slice(min_lon, max_lon), lat=slice(min_lat, max_lat))
        return da
    # Try rioxarray spatial index
    if _HAS_RIO and hasattr(da, 'rio') and da.rio.crs:
        return da.rio.clip_box(minx=min_lon, miny=min_lat, maxx=max_lon, maxy=max_lat)
    return da # fallback: no-op




def read_dsrf_chip(nc_path: Path, bbox: Tuple[float,float,float,float]) -> xr.DataArray:
    """Read DSRF variable (single band), crop to AOI; returns DataArray [y,x]."""
    ds = xr.open_dataset(nc_path, engine='netcdf4')
    # Heuristics to pick variable
    var = None
    for cand in ['DSR','dsr','dsrf','SFC_SW_DWN','Downward_Shortwave_Radiation_Flux']:
        if cand in ds.data_vars:
            var = cand; break
    if var is None:
        # take the first 2D var
        for v in ds.data_vars:
            if ds[v].ndim >= 2:
                var = v; break
    da = ds[var].squeeze()
    if bbox is not None:
        da = _crop_latlon_da(da, bbox)
    # Normalize to 0–1 roughly by a robust max (e.g. 1200 W/m^2)
    da = da.astype('float32')
    da = xr.where(np.isfinite(da), da, 0.0)
    da = da.clip(min=0)
    da = da / 1200.0
    return da

def read_mcmipf_chip(nc_path: Path, bbox: Tuple[float,float,float,float]) -> xr.DataArray:
    """Read multi-band MCMIPF. Prefer satpy for proper calibration; fallback to xarray stacking all 2D vars.
    Returns DataArray [band,y,x]."""
    try:
        scn = Scene(filenames=[str(nc_path)])
        # Load all available channels (common set); adjust if needed
        avail = sorted(scn.available_dataset_names())
        # Example: choose reflectance/BT core set
        use = [d for d in avail if any(k in d.lower() for k in ['c01','c02','c03','c07','c13','c14','c15'])]
        if not use:
            use = avail[:8]
        scn.load(use)
        # Reproject to latlon 0.02° (~2km at equator). Adjust for res.
        res_deg = 0.02
        area = scn[use[0]].attrs.get('area', None)
        if area is not None:
            tscn = scn.resample('geos') if bbox is None else scn.resample('latlon',
            radius_of_influence=6000,
            reduce_data=False,
            dst_res=(res_deg, res_deg))
        else:
            tscn = scn
        arrays = []
        for name in use:
            img = tscn[name].to_xarray()
            da = img.squeeze()
            if bbox is not None:
                da = _crop_latlon_da(da, bbox)
            arrays.append(da.astype('float32'))
        # Stack into band dimension
        da = xr.concat(arrays, dim='band')
        da = da.fillna(0.0)
        # Simple per-band standardization (robust)
        da = (da - da.quantile(0.5, dim=('y','x'))) / (da.quantile(0.9, dim=('y','x')).clip(min=1e-3))
        return da.astype('float32')
    except Exception as e:
        warnings.warn(f"satpy read failed, falling back to xarray: {e}")
        # Fallback: stack 2D variables found
    ds = xr.open_dataset(nc_path, engine='netcdf4')
    bands = []
    for v in ds.data_vars:
        if ds[v].ndim >= 2:
            da = ds[v].squeeze()
            if bbox is not None:
                da = _crop_latlon_da(da, bbox)
            bands.append(da.astype('float32'))
        if not bands:
            raise ValueError(f"No 2D vars to stack in {nc_path}")
        da = xr.concat(bands, dim='band').fillna(0.0)
        # Simple scaling
        da = (da - da.mean(dim=('y','x'))) / (da.std(dim=('y','x')) + 1e-6)
        return da

Ground: (57789, 41) (12384, 41) (12384, 41)  | Sat: (300, 32)


### Sequence builder

In [ ]:
def build_image_sequences(ts_index: pd.DatetimeIndex, file_map: pd.Series, L: int, bbox, product: str) -> Dict[pd.Timestamp, np.ndarray]:


for t in ts_index:
seq_times = pd.date_range(end=t, periods=L, freq=freq)
chips = []
ok = True
for tt in seq_times:
p = file_map.get(tt)
if p is None:
ok = False; break
if product.upper() == 'DSRF':
da = read_dsrf_chip(p, bbox)
arr = da.to_numpy()[None, ...] # [1,H,W]
else:
da = read_mcmipf_chip(p, bbox)
arr = da.to_numpy() # [B,H,W]
chips.append(arr)
if not ok:
continue
# Stack over time: [L, H, W, C]
if product.upper() == 'DSRF':
tensor = np.stack([c for c in chips], axis=0) # [L,1,H,W]
tensor = np.transpose(tensor, (0,2,3,1)) # [L,H,W,1]
else:
tensor = np.stack([np.transpose(c, (1,2,0)) for c in chips], axis=0) # each c: [B,H,W]→[H,W,B]
out[t] = tensor.astype('float32')
return out




def build_tabular_sequences(df: pd.DataFrame, target_col: str, L: int) -> Dict[pd.Timestamp, Tuple[np.ndarray, float]]:
"""Return dict target_time → (X_tab_seq [L,F] or [F] if L==1, y)"""
# Columns selection
if target_col not in df.columns:
raise KeyError(f"{target_col} not found in tabular df")
feat_cols = [c for c in df.columns if c != target_col and pd.api.types.is_numeric_dtype(df[c])]
X = df[feat_cols].astype('float32')
y = df[target_col].astype('float32')


out: Dict[pd.Timestamp, Tuple[np.ndarray, float]] = {}
idx = df.index
for i in range(L-1, len(df)):
t = idx[i]
block = X.iloc[i-L+1:i+1].values # [L,F]
if np.isnan(block).any() or not np.isfinite(y.iloc[i]):
continue
out[t] = (block, float(y.iloc[i]))
return out

=== PreDiagnosis ===
Ground test shape: (12384, 41)
Sat features shape: (300, 32)
Ground test range: 2025-01-01 04:00:00+00:00 -> 2025-03-28 03:50:00+00:00
Sat features range: 2024-02-01 00:30:00+00:00 -> 2024-02-10 15:10:00+00:00
Columnas satelitales en sat: 32
Columnas satelitales en Xte: 0
Columnas en train pero no en test: 32
Son: ['C13_std', 'C15_std', 'C06_std', 'C07_mean', 'C01_mean', 'C01_std', 'C09_mean', 'C07_std', 'C12_mean', 'C12_std', 'C05_mean', 'C11_mean', 'C16_mean', 'C13_mean', 'C04_std', 'C14_mean', 'C09_std', 'C03_mean', 'C04_mean', 'C08_mean', 'C16_std', 'C14_std', 'C08_std', 'C10_std', 'C06_mean', 'C05_std', 'C11_std', 'C02_mean', 'C02_std', 'C10_mean', 'C15_mean', 'C03_std']


### Tf

In [ ]:
def make_tf_dataset(img_dict: Dict[pd.Timestamp, np.ndarray], tab_dict: Dict[pd.Timestamp, Tuple[np.ndarray,float]],
batch: int, shuffle: bool=True) -> tf.data.Dataset:
keys = sorted(set(img_dict.keys()) & set(tab_dict.keys()))
Ximg, Xtab, Y = [], [], []
for t in keys:
xi = img_dict[t]
xt, y = tab_dict[t]
Ximg.append(xi)
Xtab.append(xt)
Y.append(y)
Ximg = np.stack(Ximg)
Xtab = np.stack(Xtab)
Y = np.array(Y, dtype='float32')
ds = tf.data.Dataset.from_tensor_slices(((Ximg, Xtab), Y))
if shuffle:
ds = ds.shuffle(min(4*batch, len(Y)))
ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
return ds, keys

Joined — train: (1440, 72) val: (12384, 40) test: (12384, 40)


## Models

In [ ]:
def build_convlstm_fusion(input_img: Tuple[int,int,int,int], input_tab: Tuple[int,int], dropout=0.2) -> tf.keras.Model:
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(32, (3,3), padding='same', return_sequences=False, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.GlobalAveragePooling2D()(x)


tab_in = layers.Input(shape=(Lt,F))
t = layers.LSTM(64, return_sequences=False)(tab_in)


h = layers.Concatenate()([x,t])
h = layers.Dropout(dropout)(h)
h = layers.Dense(128, activation='relu')(h)
h = layers.Dense(64, activation='relu')(h)
out = layers.Dense(1, dtype='float32')(h)


m = models.Model(inputs=[img_in, tab_in], outputs=out)
m.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
return m




def build_3dcnn_fusion(input_img: Tuple[int,int,int,int], input_tab: Tuple[int,int], dropout=0.2) -> tf.keras.Model:
"""3D CNN over time × H × W + MLP for tabular → fusion."""
L, H, W, C = input_img
Lt, F = input_tab
img_in = layers.Input(shape=(L,H,W,C))
x = layers.Conv3D(32, (3,3,3), strides=(1,2,2), padding='same', activation='relu')(img_in)
x = layers.BatchNormalization()(x)
x = layers.Conv3D(64, (3,3,3), strides=(1,2,2), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.GlobalAveragePooling3D()(x)


tab_in = layers.Input(shape=(Lt,F))
t = layers.Flatten()(tab_in) if Lt==1 else layers.LSTM(64)(tab_in)
t = layers.Dense(128, activation='relu')(t)


h = layers.Concatenate()([x,t])
h = layers.Dropout(dropout)(h)
h = layers.Dense(128, activation='relu')(h)
h = layers.Dense(64, activation='relu')(h)
out = layers.Dense(1, dtype='float32')(h)


m = models.Model(inputs=[img_in, tab_in], outputs=out)
m.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='mse', metrics=['mae'])
return m

Baseline → RMSE: 72215650304.0  MAE: 42850.25390625


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- C01_mean
- C01_std
- C02_mean
- C02_std
- C03_mean
- ...


## Train/Test

In [ ]:
def train_fusion(cfg: Config):
'product': cfg.product,
'seq_len': int(cfg.seq_len),
'rmse': rmse,
'mae': mae,
'skill': float(skill),
'n_test': int(len(y_true))
}
with open(cfg.out_dir / f"metrics_{cfg.product.lower()}.json", 'w') as f:
json.dump(out, f, indent=2)


# Append/Write summary CSV
summary_csv = cfg.out_dir / "fusion_test_summary.csv"
pd.DataFrame([out]).to_csv(summary_csv, mode='a' if summary_csv.exists() else 'w', index=False, header=not summary_csv.exists())


# Visualizations (guardadas en ../reports/figures)
OUT_FIG = Path("../reports/figures"); OUT_FIG.mkdir(parents=True, exist_ok=True)
N = min(400, len(y_true))
# 1) Time series clip
plt.figure(figsize=(12, 3.6))
plt.plot(idx_te[:N], y_true[:N], label="truth", lw=1.4)
plt.plot(idx_te[:N], y_pred[:N], label=f"{cfg.product} fusion", lw=1.1)
plt.plot(idx_te[:N], y_base[:N], label="baseline", lw=1.0, alpha=0.7)
plt.title(f"Test — Truth vs Fusion ({cfg.product}) vs Baseline ({cfg.target_col})")
plt.ylabel("GHI (W/m²)" if cfg.target_col.startswith("y_ghi") else "target")
plt.xlabel("Time"); plt.grid(True, ls="--", alpha=0.3); plt.legend()
plt.xticks(rotation=45); plt.tight_layout()
plt.savefig(OUT_FIG / f"{cfg.product}_fusion_ts_test.png", dpi=140)
plt.close()


# 2) Scatter
lim_min = float(min(np.min(y_true), np.min(y_pred)))
lim_max = float(max(np.max(y_true), np.max(y_pred)))
plt.figure(figsize=(4.8, 4.8))
plt.scatter(y_true, y_pred, s=10, alpha=0.5)
plt.plot([lim_min, lim_max], [lim_min, lim_max], 'r--', lw=1.0)
plt.xlabel("Actual"); plt.ylabel("Predicted")
plt.title(f"{cfg.product} Fusion — Actual vs Predicted
RMSE={rmse:.3f} MAE={mae:.3f} Skill={skill:.3f}")
plt.grid(True, ls="--", alpha=0.3); plt.tight_layout()
plt.savefig(OUT_FIG / f"{cfg.product}_fusion_scatter.png", dpi=140)
plt.close()


# 3) Residuals
resid = y_pred - y_true
plt.figure(figsize=(6, 3.2))
plt.hist(resid, bins=50, alpha=0.85)
plt.axvline(0, color='r', ls='--', lw=1)
plt.title(f"{cfg.product} Fusion — Residuals (mean={np.mean(resid):.3f})")
plt.xlabel("Residual"); plt.ylabel("Frequency")
plt.grid(True, ls="--", alpha=0.3); plt.tight_layout()
plt.savefig(OUT_FIG / f"{cfg.product}_fusion_residuals.png", dpi=140)
plt.close()


print(json.dumps(out, indent=2))


return model, out

NameError: name 'sat' is not defined

## Entry example

In [ ]:
if __name__ == "__main__":
# Example configuration — EDIT paths and AOI
cfg = Config(
product='DSRF',
dsrf_dir=Path("../goes_dsrf_nc/"),
mcmipf_dir=Path("../goes_mcmipf_nc/"),
parquet_train=Path("../data_processed/ground_train_h6.parquet"),
parquet_val=Path("../data_processed/ground_val_h6.parquet"),
parquet_test=Path("../data_processed/ground_test_h6.parquet"),
target_col='y_ghi_h6',
aoi_bbox=(-81.5, -4.5, -66.0, 13.0), # Colombia-ish bbox; adjust
seq_len=12,
lead_steps=0,
batch_size=8,
epochs=40,
lr=1e-3,
out_dir=Path("../models")
)


# Run once for DSRF
print("\n>>> Training fusion with DSRF…")
cfg.product = 'DSRF'
train_fusion(cfg)


# And once for MCMIPF (re-using the rest)
print("\n>>> Training fusion with MCMIPF…")
cfg.product = 'MCMIPF'
train_fusion(cfg)

Joined shapes — train: (40344, 40) val: (12360, 40) test: (0, 40)


In [ ]:
mdl_lstm = models.Sequential([
    layers.Input(shape=(SEQ_LEN, n_features)),
    layers.LSTM(64),
    layers.Dense(1)
])
hist_lstm, yhat_lstm = fit_and_eval(mdl_lstm, "LSTM")

In [ ]:
mdl_bilstm = models.Sequential([
    layers.Input(shape=(SEQ_LEN, n_features)),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(1)
])
hist_bilstm, yhat_bilstm = fit_and_eval(mdl_bilstm, "BiLSTM")

In [ ]:
mdl_cnnlstm = models.Sequential([
    layers.Input(shape=(SEQ_LEN, n_features)),
    layers.Conv1D(32, kernel_size=3, padding="causal", activation="relu"),
    layers.MaxPooling1D(pool_size=2),
    layers.LSTM(64),
    layers.Dense(1)
])
hist_cnnlstm, yhat_cnnlstm = fit_and_eval(mdl_cnnlstm, "CNN-LSTM")

## Comparison

In [ ]:
def plot_series(y_true, preds_dict, n=500, title="Test — Truth vs Models (primeros puntos)"):
    sl = slice(0, min(n, len(y_true)))
    plt.figure(figsize=(10,3))
    plt.plot(y_true[sl], label="truth", lw=1.2)
    for name, yhat in preds_dict.items():
        plt.plot(pd.Series(yhat, index=i_te)[sl], label=name, lw=1.0)
    plt.title(title); plt.grid(True, ls="--", alpha=0.3); plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
from math import isfinite
plot_series(pd.Series(yte_seq, index=i_te),
            {"baseline": ybase_seq,
             "RF": yhat_te_rf.reindex(i_te).values,
             "LSTM": yhat_lstm,
             "BiLSTM": yhat_bilstm,
             "CNN-LSTM": yhat_cnnlstm},
            n=500)

In [ ]:
def scatter_plot(y_true, y_pred, name):
    plt.figure(figsize=(4.2,4))
    plt.scatter(y_true, y_pred, s=8, alpha=0.4)
    lim = [0, max(2.0, np.nanmax(y_true), np.nanmax(y_pred))]
    plt.plot(lim, lim, "k--", lw=1)
    plt.xlim(lim); plt.ylim(lim)
    plt.xlabel("y_true (k)"); plt.ylabel("y_pred (k)")
    plt.title(f"Scatter Test — {name}")
    plt.grid(True, ls="--", alpha=0.3); plt.tight_layout(); plt.show()

scatter_plot(yte_seq, yhat_cnnlstm, "CNN-LSTM")

In [ ]:
# Hist residual (CNN-LSTM)
res = yhat_cnnlstm - yte_seq
plt.figure(figsize=(6,3)); plt.hist(res, bins=40, alpha=0.85)
plt.title("Residuals (y_pred - y_true) — CNN-LSTM (Test)")
plt.xlabel("residual"); plt.ylabel("count")
plt.grid(True, ls="--", alpha=0.3); plt.tight_layout(); plt.show()


In [ ]:
# Skill vs Baseline (secuencias)
def rmse_np(a,b): return np.sqrt(np.mean((a-b)**2))
rmse_base = rmse_np(yte_seq, ybase_seq)
for name, yhat in [("RF", yhat_te_rf.reindex(i_te).values),
                   ("LSTM", yhat_lstm),
                   ("BiLSTM", yhat_bilstm),
                   ("CNN-LSTM", yhat_cnnlstm)]:
    s = 1 - (rmse_np(yte_seq, yhat) / rmse_base)
    print(f"Skill (RMSE) vs baseline — {name}: {s:.3f}")